# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [21]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
#reading csv getting all data--Convert into df 
city_data=pd.read_csv("city_weather_data.csv")

city_data.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,DateTime
0,0,Tokyo,35.6895,139.6917,76.64,83,75,1.01,JP,1631454692
1,1,Jakarta,-6.2146,106.8451,82.49,74,20,6.91,ID,1631454671
2,2,Delhi,28.6667,77.2167,84.31,94,75,3.44,IN,1631454644
3,3,Mumbai,19.0144,72.8479,80.58,94,75,4.61,IN,1631454622
4,4,Manila,14.6042,120.9822,80.60,88,40,2.30,PH,1631454850


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [41]:
#config gmaps 
gmaps.configure(api_key=g_key)

#Store latitude and longitude in locations
locations=city_data[['Lat', 'Lng']]

#Fill NaN values and convert to float
humidity=city_data['Humidity'].astype(float)

#plot heatmap 
fig=gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=80,
                                 point_radius=2)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [32]:
# narrow down all data to create a new vacation df (ideal weather) ( using three conditions)
weather_df=city_data.loc[(city_data['Max Temp']>70) & (city_data['Max Temp']<80) & (city_data['Wind Speed']<10)& (city_data['Cloudiness']==0)]

ideal_weather=weather_df.dropna().reset_index(drop=True)
ideal_weather.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,DateTime
0,6,São Paulo,-23.5475,-46.6361,78.46,59,0,5.75,BR,1631454893
1,7,Seoul,37.5683,126.9778,72.97,78,0,3.44,KR,1631454742
2,20,Istanbul,41.0351,28.9833,78.22,38,0,9.22,TR,1631454913
3,29,Nanyang,32.9947,112.5328,73.24,76,0,6.91,CN,1631454927
4,39,Zhoukou,33.6333,114.6333,72.82,81,0,6.58,CN,1631454940


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [37]:
#new dataframe adding hotel name 
hotel_df=ideal_weather.loc[:,['City','Country','Lat','Lng']]
hotel_df['Hotel Name']=" "


#Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.
#base url
base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#set up params 
params = {
    "radius": 5000,
    "keyword": "hotels",
    "key": g_key,
}

#iteration 
for index, row in hotel_df.iterrows():
    #getting lat and lng 
    Lat = row['Lat']
    Lng = row['Lng']
    #adding location (must be lat, lng) into the params 
    params["location"] = f"{Lat},{Lng}"
    #getting responses , conveting to json format (using try, except )
    hotel_search=requests.get(base_url,params=params).json()
    try:
        hotel_df.loc[index,"Hotel Name"]=hotel_search['results'][0]['name']
    except:
        print("no hotel near 5000m from your location")
    #pprint(hotel_search)
hotel_df.head()
#Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

no hotel near 5000m from your location
no hotel near 5000m from your location
no hotel near 5000m from your location
no hotel near 5000m from your location


,City,Country,Lat,Lng,Hotel Name
0,São Paulo,BR,-23.5475,-46.6361,Meliá Paulista
1,Seoul,KR,37.5683,126.9778,Novotel Ambassador Seoul Yongsan - Seoul Drago...
2,Istanbul,TR,41.0351,28.9833,"Radisson Blu Hotel, Istanbul Sisli"
3,Nanyang,CN,32.9947,112.5328,Holiday Inn Nanyang
4,Zhoukou,CN,33.6333,114.6333,Zhoukou Hotel


In [38]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [40]:
# Add marker layer ontop of heat map
markers_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers_layer)
# Display figure
fig



Figure(layout=FigureLayout(height='420px'))